In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import Input
from tensorflow.data import Dataset
import pandas as pd
import numpy as np
import itertools
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('new_way.csv')
df['action'].count()

In [ ]:
x = df.drop('action',axis=1)
y = df.action

In [ ]:
lb = LabelEncoder()
y = lb.fit_transform(y)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0, stratify = y)

In [ ]:
model = Sequential([
    Input(shape=(15)),
    Dense(5, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(
    x_train,
    y_train,
    steps_per_epoch = 5,
    epochs = 40,
    validation_data = (x_test, y_test),
    verbose=1
)

In [ ]:
def plot_learningCurve(history, epochs):
  # Plot training & validation accuracy values
  epoch_range = range(1, epochs+1)
  plt.plot(epoch_range, history.history['accuracy'])
  plt.plot(epoch_range, history.history['val_accuracy'])
  plt.title('Model accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Val'], loc='upper left')
  plt.show()

  # Plot training & validation loss values
  plt.plot(epoch_range, history.history['loss'])
  plt.plot(epoch_range, history.history['val_loss'])
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Val'], loc='upper left')
  plt.show()

In [ ]:
plot_learningCurve(history, 40)

In [ ]:
test_X = np.array([
    [1321.889771,1054.833008,1278.117920,-317.187561,-117.191925,643.934631,-293.195221,-117.276314,747.565552,-186.781418,57.922749,831.953430,421.568390,282.367767,645.893616],
    [-370.724579,-181.217148,70.977104,680.856384,244.135330,425.930145,660.864990,252.465317,420.329529,582.291016,273.523987,348.257385,418.830933,280.534210,641.699524],
    [1174.421509,811.148682,1237.666992,-340.420105,-245.926758,529.321777,-366.678040,-255.753250,494.149170,418.830933,280.534210,641.699524,418.830933,280.534210,641.699524],
    [615.333618,427.486267,789.206970,328.846710,47.515835,485.224457,283.617737,39.852085,444.249359,421.568390,282.367767,645.893616,421.568390,282.367767,645.893616],
    [421.568390,282.367767,645.893616,761.635010,301.182281,498.242401,711.948303,304.129578,468.107452,421.568390,282.367767,645.893616,421.568390,282.367767,645.893616],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
])
Y = model.predict_on_batch(test_X)
print(Y)

In [ ]:
test_X = np.array([
    [-371.472626,-270.972260,-142.165512,-33.497070,-232.640335,-80.189026,160.922134,-66.725060,54.446323,238.968887,23.842035,79.471870,418.830933,280.534210,641.699524],
])
Y = model.predict(test_X)
print(Y)

In [ ]:
test_X = np.array([
        [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
])
Y = model.predict(test_X)
print(Y)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
def representative_dataset_gen():
    for _ in range(10000):
        yield [
            np.array(
                [np.random.uniform(), np.random.uniform()]
            , dtype=np.float32)
        ]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
tflite_quant_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_quant_model)

In [ ]:
# xxd -i converted_model.tflite > model_data.cc